# WorldMet Meteorological Data

In [11]:
setwd("~/Repositories/AirQualityCOVID/")

In [12]:
# Loading
suppressMessages(library(worldmet))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(openair))

In [13]:
study.prd <- c(ymd_hms("2013-01-01 00:00:00"), ymd_hms("2020-12-31 00:00:00"))

period <- as.integer(interval(floor_date(study.prd[1], unit="day"),
                    floor_date(study.prd[2], unit="day")
                   ) / (3600*24)) + 2

## Air Quality sites info

In [14]:
sites.AQ <- read.csv("data/Curation/checked_AQ.csv",
                     stringsAsFactor=TRUE)

#sites.lv <- levels(as.factor(sites.AQ$site))[1:3]
sites.lv <- c("es0118a", "es1438a") # Big cities (Madrid and Barcelona)",
sites.lv <- c(sites.lv, "es1580a", "es1340a") # small cities (Santander and Huelva)

sites.lv <- sites.lv[which(sites.lv %in% sites.AQ$site)]

In [15]:
sites.Mto <- data.frame()

for (st in sites.lv) {
    mto <- getMeta(lat = sites.AQ[sites.AQ$site == st, ]$latitude[1], 
                   lon = sites.AQ[sites.AQ$site == st, ]$longitude[1], 
                   end.year = "current",
                   n = 3, returnMap = F)

    for (cd in mto[order(mto$dist), ]$code) {
        fileName <- paste("data/Curation/WorldMet/", cd, ".csv", sep="")
        
        if (file.exists(fileName)) {
            mto[mto$code == cd, "siteAQ"] <- st
            sites.Mto <- rbind(sites.Mto, mto[mto$code == cd, ])            
            
            break
        } else {
            data <- importNOAA(code = cd,
                              year = 2013:2020,
                              hourly = TRUE,
                              n.cores = 6
                             ) %>%
                        select("date", "code", "ws", "wd", "atmos_pres") %>%
                        timeAverage(avg.time = "day", type="code")

            if(sum(colSums(!is.na(data))/ period > 0.8) == ncol(data)) {
                mto[mto$code == cd, "siteAQ"] <- st
                sites.Mto <- rbind(sites.Mto, mto[mto$code == cd, ])
                
                #write.csv(data, fileName, row.names=F)

                break
            }
        }
    }
}

[1] "The following sites / years are missing:"
          code year date station
1 082220-99999 2013 <NA>    <NA>
2 082220-99999 2014 <NA>    <NA>
3 082220-99999 2015 <NA>    <NA>
4 082220-99999 2016 <NA>    <NA>
[1] "The following sites / years are missing:"
          code year date station
1 082200-99999 2013 <NA>    <NA>
2 082200-99999 2014 <NA>    <NA>
3 082200-99999 2015 <NA>    <NA>
4 082200-99999 2016 <NA>    <NA>
[1] "The following sites / years are missing:"
          code year date station
1 081800-99999 2013 <NA>    <NA>
2 081800-99999 2014 <NA>    <NA>
3 081800-99999 2015 <NA>    <NA>
4 081800-99999 2016 <NA>    <NA>


## Save Data to CSV 

```R
write.csv(sites.Mto,
          paste("data/Curation/checked_WorldMet.csv"), row.names=FALSE)
```